# Project phase 1: Baseline

The goal of this phase is to create a baseline model. Note that the word baseline can mean different things. In the course we distinguished three different types of baselines:
* 1. The simplest possible approach (majority baseline, i.e. everything is positive or noun)
* 2. A simple machine learning classifier (logistic regression with words as features)
* 3. The ``state-of-the-art'' approach on which you want to improve (your starting point)

For this phase you need to make a number 2 or 3 baseline. 

If you plan to have a research question like: can we improve sentiment detection systems by doing X, the answer to the question is the most relevant if you have a competetive baseline (3). In this case we would suggest to use a BiLSTM or even a transformer based model, so that you can re-use the baseline for the final research question (phase 3).

You should pick one of the following tasks to create your baseline for.

## Task 1: Sentiment classification
* The data can be found in the `classification` folder.
* The goal is to predict the label in the `sentiment` field.
* **You have to upload the predictions of `music_reviews_test_masked.json.gz` to CodaLab. (The link will be posted here on monday). Note that the format should match the json files in the repository.**
* **Also upload a .txt file on LearnIt (one per group) with a short description of your baseline.**

The data can be read like:

In [ ]:
import numpy as np 
import torch 
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
import gzip
import json
for line in gzip.open('classification/music_reviews_dev.json.gz'):
    review_data = json.loads(line)
    for key in review_data:
        print('"' + key +'": ' + str(review_data[key]))
    break
paths = {'train':'classification/music_reviews_train.json.gz',
        'test':'classification/music_reviews_test_masked.json.gz'}

In [ ]:
train_vocab = {}
train = gzip.open(paths['train'])
counter1 = 0
counter2 = 0
counter3 = 0
train_no_reviewText = []
labels = {}
train_sentences = {}
for line in train:
    counter1 +=1
    #print(line)
    if 'reviewText' in json.loads(line).keys():
        train_sentences[counter3] = json.loads(line)['reviewText']
        counter3 += 1
        for word in json.loads(line)['reviewText'].split():
            if word not in train_vocab.keys():
                train_vocab[word] = counter2
                counter2 += 1
    else:
        train_no_reviewText.append(counter1)
print(counter3,counter2)

In [ ]:
# Gram matrix 
m1 = torch.zeros(counter3, counter2)


In [ ]:
# Need idx2word
idx2word = dict([(value, key) for key, value in train_vocab.items()])

# Begin correcting gram matrix

for sen in train_sentences: 
    for word in train_sentences[sen].split(): 
        m1[sen, train_vocab[word]] = 1

In [ ]:
train_vocab = {}
train = gzip.open(paths['train'])
train_labels = {}
counter = 0
for line in train:
    a = json.loads(line)
    if 'reviewText' in a.keys():
        if a['sentiment'] == 'positive':
            train_labels[counter] = 1
        elif a['sentiment'] == 'negative': 
            train_labels[counter] = 0
        counter +=1

        
#len(labels)
#print(type(labels))



counter2

In [ ]:
batch_size = 5000
num_batches = int(len(m1)/batch_size)
train_feats_batches = m1[:batch_size*num_batches].view(num_batches,batch_size, counter2)
for feats_batch in train_feats_batches:
    print(feats_batch.shape)

bingus = list(train_labels.values())
bingus = torch.FloatTensor(bingus)

num_batches = int(len(bingus)/batch_size)
train_label_batches = bingus[:batch_size*num_batches].view(num_batches,batch_size,1)
counter = 1
for feats_batch in train_label_batches:
    counter+=1
    print(feats_batch.shape)
counter

In [ ]:
# Encode test labels
test_vocab = {}
test = gzip.open(paths['test'])
counter1 = 0
counter2 = 0
counter3 = 0
test_no_reviewText = []
test_labels = {}
test_sentences = {}
for line in test:
    counter1 +=1
    #print(line)
    if 'reviewText' in json.loads(line).keys():
        test_sentences[counter3] = json.loads(line)['reviewText']
        counter3 += 1
        for word in json.loads(line)['reviewText'].split():
            if word not in train_vocab.keys():
                test_vocab[word] = counter2
                counter2 += 1
    else:
        test_no_reviewText.append(counter1)
print('Vocab size: ', counter2)
        
# Construct gram matrix
m2 = torch.zeros(counter3, 226347)
print('m2 constructed!')


# Need idx2word
idx2word = dict([(value, key) for key, value in train_vocab.items()])
print('idx2word done!')


# Begin correcting gram matrix
for sen in test_sentences: 
    for word in test_sentences[sen].split(): 
        if word in train_vocab.keys():
            m2[sen, train_vocab[word]] = 1
print('Gram matrix done')

#Note labels
test = gzip.open(paths['test'])
test_labels = {}
counter = 0
for line in test:
    a = json.loads(line)
    if 'reviewText' in a.keys():
        if a['sentiment'] == 'positive':
            test_labels[counter] = 1
        elif a['sentiment'] == 'negative': 
            test_labels[counter] = 0
        counter +=1
print('Labels noted!')
        
#Divide into batches

batch_size = 2499
num_batches = int(len(m2)/batch_size)
test_feats_batches = m2[:batch_size*num_batches].view(num_batches,batch_size, 226347)
print('Feature Matrix shapes: ')
for feats_batch in test_feats_batches:
    print(feats_batch.shape)
bingus = list(test_labels.values())
bingus = torch.FloatTensor(bingus)
num_batches = int(len(bingus)/batch_size)
test_label_batches = bingus[:batch_size*num_batches].view(num_batches,batch_size,1)
print('label matrix shapes: ')
for feats_batch in test_label_batches:
    print(feats_batch.shape)

In [ ]:
def load_vocab(filepath):
    test_vocab = {}
    test = gzip.open(filepath)
    counter1 = 0
    counter2 = 0
    counter3 = 0
    test_no_reviewText = []
    test_labels = {}
    test_sentences = {}
    for line in test:
        counter1 +=1
        #print(line)
        if 'reviewText' in json.loads(line).keys():
            test_sentences[counter3] = json.loads(line)['reviewText']
            counter3 += 1
            for word in json.loads(line)['reviewText'].split():
                if word not in train_vocab.keys():
                    test_vocab[word] = counter2
                    counter2 += 1
        else:
            test_no_reviewText.append(counter1)
    final_dict = {'line_count' : counter1,
                 'review_count' : counter3,
                 'vocab_size' : counter2,
                 'no_text_reviews' : test_no_reviewText,
                 'labels' : test_labels,
                 'vocabulary' : test_vocab,
                 'sentences' : test_sentences}
    return final_dict

def construct_gram(vocab, num_sen, vocab_len, sentences): 
    # Construct gram matrix
    m2 = torch.zeros(num_sen, vocab_len)
    print('m2 constructed!')
    for sen in sentences: 
        for word in sentences[sen].split(): 
            m2[sen, vocab[word]] = 1
    print('Gram matrix done')
    return m2



In [ ]:
#model = LogisticRegression()
#for feat, label in zip(train_feats_batches,train_label_batches):
#    model.fit(feat,label)


In [ ]:
preds = {}
counter = 0
for batch in test_feats_batches:
    preds[counter] = model.predict(batch)
    counter+=1

In [ ]:
new_arr = []
for i in preds:
    for j in preds[i]:
        new_arr.append(j)
len(new_arr)
new_arr = np.array(new_arr)
new_arr.shape

In [ ]:
for pred, true in zip(preds, test_label_batches):
    print(classification_report(preds[pred], true))

In [ ]:
test = gzip.open(paths['test'])
counter = 0
new_data = []
for i in test:
    bingus = json.loads(i)
    if new_arr[counter] == 0:
        bingus['sentiment'] = 'negative'
    elif new_arr[counter] == 1:
        bingus['sentiment'] = 'positive'
    new_data.append(bingus)

In [ ]:
for i in new_data:
    print(i)
    break

In [ ]:
#with open("final.json", 'a') as f:
#    for i in new_data:
#        json.dump(i,f)
#        f.write('\n')

# Part 2: Break it down

In this part of the project, we are tasked with breaking our own model down, to try and improve it. <br> 

### Suggested methods: 
- Change language
- More negation
- Reviews of other products

### Things we should also consider: 
- Better tokenization
- Model tuning
- Acutually using development data

In [ ]:
# better tokenization: 
'''
1. Look up better regex expression
2. Remove stopwords
3. implement padding(might have to wait on that one lmao)
4. Use a way more sophisticated model. (might wanna wait on that one too)
'''













In [ ]:
#Reopen file + imports: 
import gzip
import json
import torch 
from nltk.tokenize import TweetTokenizer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import classification_report

# This is how they load the data ------------------------------------------
#for line in gzip.open('../classification/music_reviews_dev.json.gz'):
#    review_data = json.loads(line)
#    for key in review_data:
#        print('"' + key +'": ' + str(review_data[key]))
#    break
#--------------------------------------------------------------------------


paths = {'train':'../classification/music_reviews_train.json.gz',
        'test':'../classification/music_reviews_test_masked.json.gz',
        'dev' : '../classification/music_reviews_dev.json.gz'}

In [ ]:
# Vocabs built using TweetTokenizer
# 


def build_vocab(filepath):
    train_vocab = {}
    train = gzip.open(filepath)
    counter1 = 0
    counter2 = 0
    counter3 = 0
    counter = 0
    no_reviewText = []
    labels = {}
    sentences = {}
    tokenizer = TweetTokenizer()
    for line in train:
        counter1 +=1
        #print(line)
        if 'reviewText' in json.loads(line).keys():
            a = json.loads(line)
            sentences[counter3] = a['reviewText']
            counter3 += 1
            if a['sentiment'] == 'positive':
                labels[counter] = 1
            elif a['sentiment'] == 'negative': 
                labels[counter] = 0
            counter +=1
            for word in tokenizer.tokenize(json.loads(line)['reviewText']):
                if word not in train_vocab.keys():
                    train_vocab[word] = counter2
                    counter2 += 1
        else:
            no_reviewText.append(counter1)
    final_dict = {'line_count' : counter1,
                 'review_count' : counter3,
                 'vocab_size' : counter2,
                 'no_text_reviews' : no_reviewText,
                 'labels' : labels,
                 'vocabulary' : train_vocab,
                 'sentences' : sentences}
    return final_dict

train_set =  build_vocab(paths['train'])
dev_set = build_vocab(paths['dev'])



In [ ]:
# Might wanna run tests again just to see if the new tokenization improved performance in any way. 
tokenizer = TweetTokenizer()
# unigrams : 
def create_unigram(vocab, sentences, tokenzier):
    # Create matrix
    m1 = torch.zeros(len(sentences), len(vocab))
    # Correct indices
    for sen in range(len(sentences)): 
        for word in tokenizer.tokenize(sentences[sen]): 
            if word in vocab.keys():
                m1[sen, vocab[word]] = 1
    return m1

train_unigram = create_unigram(train_set['vocabulary'], train_set['sentences'], tokenizer)

def create_batches(matrix, batch_size,labels): 
    num_batches = int(len(matrix)/batch_size)
    feats_batches = matrix[:batch_size*num_batches].view(num_batches,batch_size, matrix.shape[1])
    bingus = torch.FloatTensor(list(labels.values()))
    num_batches = int(len(bingus)/batch_size)
    label_batches = bingus[:batch_size*num_batches].view(num_batches,batch_size,1)
    return feats_batches, label_batches
train_feat_batches = create_batches(train_unigram, 2499, train_set['labels'])

In [ ]:
labels_batches = list(train_feat_batches[1])
feat_batches = list(train_feat_batches[0])

In [ ]:
# Make model
model = LogisticRegression()

# Train model

for feat, label in zip(feat_batches, labels_batches):
    model.fit(feat,label)

In [ ]:
dev_unigram = create_unigram(train_set['vocabulary'], dev_set['sentences'], tokenizer)

In [ ]:
dev_batches = create_batches(dev_unigram,2499,dev_set['labels'])
preds = {}
counter = 0
for batch in dev_batches[0]:
    preds[counter] = model.predict(batch)
    counter+=1
new_arr = []
for i in preds:
    for j in preds[i]:
        new_arr.append(j)
len(new_arr)
new_arr = np.array(new_arr)
for pred, true in zip(preds, dev_batches[1]):
    print(classification_report(preds[pred], true))

In [ ]:
hard_preds = {}
counter = 0
for batch, pred_dict, true_dict in zip(dev_batches[0], preds.values(), dev_batches[1]):
    for sentence, pred, true in zip(batch, pred_dict, true_dict):
        if pred != true:
            hard_preds[counter] = int(pred)
        counter += 1

In [ ]:
hard_preds

In [ ]:
# grabbing the first 200 misclassification sentences
hard_sens = []
for i in hard_preds:
    hard_sentence = train_set['sentences'][i]
    if len(hard_sens) >= 200:
        break
    else:
        hard_sens.append(i)

In [ ]:
test = gzip.open(paths['dev'])
new_data = []
counter = 0
for i in test:
    bingus = json.loads(i)
    if counter in hard_sens:
        new_data.append(bingus)
    counter+=1

In [ ]:
with open("hard_cases.json", 'a') as f:
    for i in new_data:
        json.dump(i,f)
        f.write('\n')

### Checklist section:
Gonna follow how they do in the official documentation.
Pretty dumb donkey way of doing this. But hope it works.


In [ ]:
# Load the same stuff that they do in the doc. 
import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.test_suite import TestSuite
from checklist.perturb import Perturb
from spacy.lang.en.examples import sentences 
import random 
# Load editor. 

editor = checklist.editor.Editor()
editor.tg
nlp = spacy.load('en_core_web_sm')
sentences = dev_set['sentences'].values()
parsed_data = list(nlp.pipe(sentences))
suite = TestSuite()

In [ ]:
parsed_data[0]

We need at least 100 samples. What we can do is define some categories of difficult sentences for our logistic regression:

 - Negation: A negation such as "I don't like the artist" is difficult for our classifier to classify correctly, as it will see the work "like" which typically has positive connotations, but in this example the word "don't" infers that we means that like is used to infer negative sentiment of the artist, which our baseline logistic regression won't be able to detect.

 - Irony/Sarcasm. Since our baseline model basically just looks at individual words and learns if they are typically positive or negative, if a reviewer is being ironic/sarcastic and using words which indicate the opposite sentiment of what they truly mean, our logistic regression will fail. Fx with the sentence "This album rocks, I love bleeding from my ears!", it uses very positive words but since they are being ironic/sarcastic they use the positive words to convey a negative sentiment.

 - pos/neg. If a reviewer uses a mix of both positive and negative reviews, our logistic regression might become a bit confused and not know with any strong confidence which sentiment the review is. Fx "I liked the old album but this new one sucks" is hard because there is both the positive work "liked" and the negative word "sucks", and our logistic regression is too simple to understand that in this context it is the reviewed album that is bad.

With these 3 categories, since we need at least 100 samples we can try to create some templates for them and generate say 40 samples each to get 120 total samples.

In [ ]:
####### Creating lists of stuff for use in generating sentences #######

# Constructing collection of music_nouns for use in all categories to reference whatever is being reviewed
music_noun = ['project', 'artist', 'album', 'genre', 'compilation', 'ep',
              'singer', 'band', 'guitar', 'drummer', 'guitarist', 'pianist',
              'group']

# Negations
negations = ['don\'t', 'can\'t' , 'not', 'won\'t', 'nothing']
past_negations = ['didn\'t', 'wouldn\'t', 'couldn\'t', 'shouldn\'t']

# Pos/Neg Adjectives
pos_adj = ['adorable', 'amazing', 'awesome', 'beautiful', 'brilliant', 'captivating',
           'creative', 'elegant', 'energetic' , 'excellent', 'exceptional', 'exciting',
           'extraordinary', 'fabulous', 'fantastic', 'fun', 'good', 'great', 'happy',
           'imaginative', 'incredible', 'nice', 'perfect', 'sweet', 'wonderful']
neg_adj = ['abrasive', 'annoying', 'average', 'awful', 'bad', 'boring', 'careless',
           'creepy', 'difficult', 'dreadful', 'frustrating', 'hard', 'horrible',
           'lame', 'lousy', 'nasty', 'poor', 'ridiculous', 'rough', 'sad', 'terrible',
           'ugly', 'unhappy', 'unpleasant', 'weird']

# Positive Verbs in Present and Past Tenses
pos_verb_present = ['admire', 'appreciate', 'enjoy', 'like', 'love', 'recommend', 'value', 'welcome']
pos_verb_past = ['admired', 'appreciated', 'enjoyed', 'liked', 'loved', 'recommended', 'valued', 'welcomed']

# Negative Verbs in Present and Past Tenses
neg_verb_present = ['abhor', 'despise', 'dislike', 'dread', 'hate', 'loathe', 'regret', 'resent']
neg_verb_past = ['abhorred', 'despised', 'disliked', 'dreaded', 'hated', 'loathed', 'regretted', 'resented']

# Neutral Verbs in Present and Past Tenses
neutral_verb_present = ['find', 'see']
neutral_verb_past = ['found', 'saw']



####### adding lexicons #######
editor.add_lexicon('music_noun', music_noun, overwrite=True)

editor.add_lexicon('negation', negations, overwrite=True)
editor.add_lexicon('past_negation', past_negations, overwrite=True)

editor.add_lexicon('pos_adj', pos_adj, overwrite=True)
editor.add_lexicon('neg_adj', neg_adj, overwrite=True)

editor.add_lexicon('pos_verb_present', pos_verb_present, overwrite=True)
editor.add_lexicon('pos_verb_past', pos_verb_past, overwrite=True)

editor.add_lexicon('neg_verb_present', neg_verb_present, overwrite=True)
editor.add_lexicon('neg_verb_past', neg_verb_past, overwrite=True)

editor.add_lexicon('neutral_verb_present', neutral_verb_present, overwrite=True)
editor.add_lexicon('neutral_verb_past', neutral_verb_past, overwrite=True)

# mixed lexicons? not too sure how these work - Aidan
editor.add_lexicon('pos_verb', pos_verb_present + pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb', neg_verb_present + neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb', neutral_verb_present + neutral_verb_past, overwrite=True)

In [ ]:
# to get 40 negations can create 2 templates for negations and randomly sample 20 sentences for each template
template_negation_neg = list(editor.template('I {negation} {pos_verb_present} the {music_noun}')['data']) # template 1
template_negation_pos = list(editor.template('I {negation} {neg_verb_present} the {music_noun}')['data']) # template 2

In [ ]:
negation_neg = random.sample(template_negation_neg, 20)
negation_neg

In [ ]:
negation_pos = random.sample(template_negation_pos, 20)
negation_pos

In [ ]:
# Irony/Sarcasm Templates
template_ironsarc_pos = list(editor.template("Sooooo totally didn't {pos_verb_present} this {music_noun}...")['data'])
template_ironsarc_neg = list(editor.template("Sooooo totally didn't {neg_verb_present} this {music_noun}...")['data'])

In [ ]:
# negative sentiment
ironsarc_neg = random.sample(template_ironsarc_neg, 20)
ironsarc_neg

In [ ]:
# positive sentiment
ironsarc_pos = random.sample(template_ironsarc_pos, 20)
ironsarc_pos

In [ ]:
# pos/neg templates
template_posneg_pos = list(editor.template("I {neg_verb_past} the old {music_noun}, but this new one I {pos_verb_present}")['data'])
template_posneg_neg = list(editor.template("I {pos_verb_past} the old {music_noun}, but this new one I {neg_verb_present}")['data'])

In [ ]:
posneg_pos = random.sample(template_posneg_pos, 20)
posneg_pos

In [ ]:
posneg_neg = random.sample(template_posneg_neg, 20)
posneg_neg

### Saving generated sentences to json file format

In [ ]:
# getting them all in a list for iteration:
generated = [negation_neg, negation_pos, ironsarc_neg, ironsarc_pos, posneg_neg, posneg_pos]

# creating json file so that if rerun this code always overwrites the existing file
with open("hard_cases.json", 'w') as f:
    f.write('')

# looping through and adding them all to json file
for category in generated:
    # getting to know categories and sentiments:
    if category == negation_neg:
        category_name = 'Negation'
        sentiment = 'Negative'
    elif category == negation_pos:
        category_name = 'Negation'
        sentiment = 'Positive'
    elif category == ironsarc_neg:
        category_name = 'Irony/Sarcasm'
        sentiment = 'Negative'
    elif category == ironsarc_pos:
        category_name = 'Irony/Sarcasm'
        sentiment = 'Positive'
    elif category == posneg_neg:
        category_name = 'Pos/Neg'
        sentiment = 'Negative'
    elif category == posneg_pos:
        category_name = 'Pos/Neg'
        sentiment = 'Positive'
    
    # adding to json file
    with open("hard_cases.json", 'a') as f:
        for reviewText in category:
            # writing all the stuff
            json.dump({'reviewText': reviewText, 'sentiment': sentiment, 'category': category_name}, f)
            f.write('\n')

In [ ]:
sen_1 = list(editor.template('I {negation} {pos_verb_present} the {music_noun}')['data'])
sen_2 = list(editor.template('I {past_negation} {neg_verb_present} the really {neg_adj} {music_noun}')['data'])
irony_1 = list(editor.template('No the {neg_adj} {music_noun}, was totally {pos_adj} yeah...')['data'])
negation_3 = list(editor.template('I actually {past_negation} {neg_verb_present} the {music_noun}')['data'])
negation_4 = list(editor.template('Contrary to what i thought, the {music_noun}, wasn\'t {neg_adj}')['data'])

In [ ]:
samples_1 = random.sample(sen_1, 15)
samples_2 = random.sample(sen_2, 15)
samples_3 = random.sample(irony_1, 15)
samples_4 = random.sample(negation_3, 15)
samples_5 = random.sample(negation_4, 15)

In [ ]:
samples_1

In [ ]:
samples_2

In [ ]:
samples_3

In [ ]:
samples_4

In [ ]:
samples_5